In [1]:
import numpy as np
import scipy.sparse as sps
import porepy as pp

In [2]:
import sys
sys.path.append("./class")
from Transport_class import Transport

In [3]:
Nx=100
#Nx=5
#Ny=20
phys_dims=[1.0]
#phys_dims=[0.045]
#phys_dims=[1,1]
#g=pp.CartGrid([Nx,Ny],phys_dims)
g=pp.CartGrid([Nx],phys_dims)
g.compute_geometry()            

In [4]:
unity = np.ones(g.num_cells)
empty = np.empty(0)
#porosity=0.3*unity
porosity=unity
aperture=1
bc_type=["dir","dir"]
#bc_value=[0.,0.]
bc_value=[0.38553623, 0.]
#init_cond=lambda x,y,z:int(x<0.5)
#init_cond=lambda x,y,z:int(x<0.2)
init_cond=lambda x,y,z:0.0 
print(g.num_faces)

101


In [5]:
specified_parameters = {
            "bc_type": bc_type,
            "bc_value": bc_value,
            #"time_step": 34.56,
            "time_step": 0.01,
            "mass_weight": porosity * aperture,
            "darcy_flux":np.ones(g.num_faces),
            #"darcy_flux":6.67e-6*np.ones(Nx+1),
            "t_max": 10,
            #"t_max":345600,
            "method": "Implicit",
            "lambda_lin_decay":0,
            "initial_cond":init_cond,
}

In [6]:
tracer=np.zeros(g.num_cells)
transport=Transport(g,specified_parameters)
#transport.set_bc()
#transport.set_initial_cond(tracer)
#lhs,rhs_b,rhs_matrix=transport.get_transport_lhs_rhs()
lhs,rhs_b,rhs_matrix=transport.set_and_get_matrices(tracer)
print("tracer=")
print(tracer)
print("lhs")
print(lhs.shape)
print(lhs)
print("rhs_b")
print(rhs_b)
print("rhs_matrix")
print(rhs_matrix)

tracer=
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
lhs
(100, 100)
  (0, 0)	2.0
  (1, 0)	-1.0
  (1, 1)	2.0
  (2, 1)	-1.0
  (2, 2)	2.0
  (3, 2)	-1.0
  (3, 3)	2.0
  (4, 3)	-1.0
  (4, 4)	2.0
  (5, 4)	-1.0
  (5, 5)	1.9999999999999996
  (6, 5)	-1.0
  (6, 6)	2.000000000000001
  (7, 6)	-1.0
  (7, 7)	1.9999999999999996
  (8, 7)	-1.0
  (8, 8)	1.9999999999999996
  (9, 8)	-1.0
  (9, 9)	2.000000000000001
  (10, 9)	-1.0
  (10, 10)	1.9999999999999996
  (11, 10)	-1.0
  (11, 11)	1.9999999999999996
  (12, 11)	-1.0
  (12, 12)	2.000000000000001
  :	:
  (87, 87)	2.000000000000001
  (88, 87)	-1.0
  (88, 88)	2.000000000000001
  (89, 88)	-1.0
  (89, 89)	2.000000000000001
  (90, 89)	-1.0
  (90, 90)	2.000000000000001
  (91, 90)	-1.0
  (91, 91)	2.0000000

In [7]:
IEsolver = sps.linalg.factorized(lhs)

data_transport=transport.data[pp.PARAMETERS]["transport"]
n_steps = int(np.round(data_transport["t_max"] / data_transport["time_step"]))

save_every=1

# Exporter
exporter = pp.Exporter(transport.g, file_name="tracer",folder_name="transport_1D")
    
for i in range(1,n_steps+1,1):
#for i in range(1,2,1):
    if np.isclose(i % save_every, 0):
        # Export existing solution (final export is taken care of below)
        exporter.write_vtu({"tracer":tracer}, time_step=int(i // save_every))
        if data_transport["method"]=="Explicit":
            tracer = IEsolver(rhs_matrix*tracer+rhs_b)
            print("i=")
            print(i)
            print(tracer)
        else:
            tracer = IEsolver(rhs_matrix*tracer+rhs_b)
            print("i=")
            print(i)
            print(tracer)
print(tracer)
exporter.write_vtu({"tracer":tracer}, time_step=(n_steps // save_every))
time_steps = np.arange(0,data_transport["t_max"] + data_transport["time_step"], save_every * data_transport["time_step"])
exporter.write_pvd(time_steps)

i=
1
[1.92768115e-01 9.63840575e-02 4.81920287e-02 2.40960144e-02
 1.20480072e-02 6.02400359e-03 3.01200180e-03 1.50600090e-03
 7.53000449e-04 3.76500225e-04 1.88250112e-04 9.41250562e-05
 4.70625281e-05 2.35312640e-05 1.17656320e-05 5.88281601e-06
 2.94140800e-06 1.47070400e-06 7.35352001e-07 3.67676001e-07
 1.83838000e-07 9.19190001e-08 4.59595001e-08 2.29797500e-08
 1.14898750e-08 5.74493751e-09 2.87246875e-09 1.43623438e-09
 7.18117189e-10 3.59058594e-10 1.79529297e-10 8.97646486e-11
 4.48823243e-11 2.24411621e-11 1.12205811e-11 5.61029054e-12
 2.80514527e-12 1.40257263e-12 7.01286317e-13 3.50643159e-13
 1.75321579e-13 8.76607896e-14 4.38303948e-14 2.19151974e-14
 1.09575987e-14 5.47879935e-15 2.73939968e-15 1.36969984e-15
 6.84849919e-16 3.42424959e-16 1.71212480e-16 8.56062399e-17
 4.28031199e-17 2.14015600e-17 1.07007800e-17 5.35038999e-18
 2.67519500e-18 1.33759750e-18 6.68798749e-19 3.34399375e-19
 1.67199687e-19 8.35998436e-20 4.17999218e-20 2.08999609e-20
 1.04499805e-20 5.2

i=
94
[0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553622 0.38553622 0.38553621 0.38553619
 0.38553616 0.3855361  0.385536   0.38553583 0.38553555 0.3855351
 0.38553437 0.38553321 0.3855314  0.38552864 0.38552446 0.38551824
 0.3855091  0.38549585 0.38547691 0.38545017 0.38541289 0.38536154
 0.38529166 0.38519762 0.38507252 0.38490792 0.38469365 0.38441765
 0.38406574 0.38362153 0.38306626 0.3823788  0.38153557 0.38051073
 0.37927626 0.37780227 0.37605732 0.37400891 0.37162397 0.36886953
 0.3657134  0.36212492 0.35807577 0.35354071 0.34849845 0.34293231
 0.33683097 0.330189   0.32300737 0.31529377 0.30706279 0.29833597
 0.28914165 0.27951465 0.26949585 0.25913158 0.24847286 0

/home/dell/.local/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


i=
103
[0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553622 0.38553622 0.3855362  0.38553618
 0.38553615 0.38553609 0.38553599 0.38553583 0.38553556 0.38553515
 0.3855345  0.38553351 0.38553199 0.38552972 0.38552636 0.38552144
 0.38551434 0.3855042  0.38548989 0.38546995 0.38544243 0.38540488
 0.3853542  0.38528648 0.38519692 0.38507964 0.38492754 0.38473214
 0.38448346 0.38416983 0.38377778 0.38329199 0.38269516 0.38196803
 0.38108942 0.38003629 0.37878392 0.37730612 0.37557554 0.37356403
 0.37124304 0.3685842  0.36555976 0.36214327 0.35831013 0.35403826
 0.34930869 0.34410616 0.33841968 0.33224298 0.32557495

i=
203
[0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623

...

i=
673
[0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623

i=
778
[0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623

i=
788
[0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623

i=
887
[0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623

i=
903
[0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623
 0.38553623 0.38553623 0.38553623 0.38553623 0.38553623